In [1]:
import json
import re
import sys
from PyQt5.QtWidgets import (
    QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton,
    QCheckBox, QListWidget, QScrollArea, QApplication,
    QComboBox, QHBoxLayout, QMainWindow, QStackedWidget, QWidgetItem
)

In [2]:
# Import python file
def load_courses():
    try:
        with open("class dictionary.json") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

def save_courses():
    with open("class dictionary.json", "w") as f:
        json.dump(courses, f, indent=2)

courses = load_courses()

In [3]:
# Function that gets all unique tags present in the dataset
# Useful when loading the graphical user interface to provide a dropdown instead of a search
def get_all_tags():
    tags = set()
    for key in courses:
        for tag in courses[key]['Tags']:
            tags.add(tag)
    return tags

In [4]:
def search_by_tag(tag):
    if type(tag) is not str:
        return []
    
    results = []
    for code, info in courses.items():
        for local_tag in info['Tags']:
            if re.search(tag.lower(), local_tag.lower()):
                results.append((code, info))
                break  # Only match once per course
    return results


In [5]:
all_tags = get_all_tags()

class StartScreen(QWidget):
    def __init__(self, stacked_widget):
        super().__init__()
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.stacked_widget = stacked_widget

        self.layout.addWidget(QLabel("Select Mode:"))
        student_button = QPushButton("Student")
        admin_button = QPushButton("Admin")
        self.layout.addWidget(student_button)
        self.layout.addWidget(admin_button)

        student_button.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(1))
        admin_button.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(2))


class StudentView(QWidget):
    def __init__(self, stacked_widget):
        super().__init__()
        self.stacked_widget = stacked_widget

        self.setWindowTitle("UniTag Course Search")
        self.setGeometry(100, 100, 700, 600)
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        self.welcome_label = QLabel("Welcome to UniTag!\nPlease select the areas you are interested in:")
        self.layout.addWidget(self.welcome_label)

        # Tag checkboxes
        checkbox_container = QWidget()
        checkbox_layout = QVBoxLayout()
        checkbox_container.setLayout(checkbox_layout)
        self.checkboxes = []
        for tag in all_tags:
            cb = QCheckBox(tag)
            self.checkboxes.append(cb)
            checkbox_layout.addWidget(cb)

        scroll = QScrollArea()
        scroll.setWidgetResizable(True)
        scroll.setWidget(checkbox_container)
        self.layout.addWidget(scroll)

        # Buttons
        self.search_button = QPushButton("Search")
        self.search_button.clicked.connect(self.perform_search)
        self.layout.addWidget(self.search_button)

        self.return_button = QPushButton("Return to Menu")
        self.return_button.clicked.connect(lambda: self.stacked_widget.setCurrentIndex(0))
        self.layout.addWidget(self.return_button)

        self.result_list = QListWidget()
        self.layout.addWidget(self.result_list)

    def perform_search(self):
        selected_tags = [cb.text() for cb in self.checkboxes if cb.isChecked()]

        matched_courses = {}
        if not selected_tags:
            # No tags selected: show all courses
            for code, info in courses.items():
                matched_courses[code] = info
        else:
            # Tag-based fuzzy matching
            for tag in selected_tags:
                results = search_by_tag(tag)
                for code, info in results:
                    matched_courses[code] = info  # deduplicates by course code

        self.result_list.clear()
        if matched_courses:
            for code, info in matched_courses.items():
                course_str = f"{code}: {info['Course Name']} ({', '.join(info['Tags'])})"
                self.result_list.addItem(course_str)
        else:
            self.result_list.addItem("No courses match the selected areas.")



In [ ]:
class AdminPanel(QWidget):
    def __init__(self, stacked_widget):
        super().__init__()
        self.setWindowTitle("Admin Panel")
        layout = QVBoxLayout()
        self.setLayout(layout)

        layout.addWidget(QLabel("Add New Course:"))
        self.dept_input = QLineEdit(); self.dept_input.setPlaceholderText("Department")
        self.crn_input = QLineEdit(); self.crn_input.setPlaceholderText("CRN")
        self.subj_input = QLineEdit(); self.subj_input.setPlaceholderText("Subject")
        self.num_input = QLineEdit(); self.num_input.setPlaceholderText("Course Number")
        self.name_input = QLineEdit(); self.name_input.setPlaceholderText("Course Name")
        self.tag_input = QLineEdit(); self.tag_input.setPlaceholderText("Comma-separated Tags")

        for field in [self.dept_input, self.crn_input, self.subj_input, self.num_input, self.name_input, self.tag_input]:
            layout.addWidget(field)

        self.course_submit = QPushButton("Add Course")
        layout.addWidget(self.course_submit)
        self.course_submit.clicked.connect(self.add_course)

        self.return_button = QPushButton("Return to Menu")
        self.return_button.clicked.connect(lambda: stacked_widget.setCurrentIndex(0))
        layout.addWidget(self.return_button)

    def add_course(self):
        department = self.dept_input.text().strip()
        crn = self.crn_input.text().strip()
        subject = self.subj_input.text().strip()
        course_number = self.num_input.text().strip()
        course_name = self.name_input.text().strip()
        tags = [t.strip() for t in self.tag_input.text().split(',') if t.strip()]

        if not (department and crn and subject and course_number and course_name and tags):
            return 

        course_key = f"{subject} {course_number}"
        courses[course_key] = {
            "Department": department,
            "CRN": crn,
            "Subject": subject,
            "Course Number": course_number,
            "Course Name": course_name,
            "Tags": tags
        }

        save_courses()

        for field in [self.dept_input, self.crn_input, self.subj_input, self.num_input, self.name_input, self.tag_input]:
            field.clear()

In [ ]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    stacked_widget = QStackedWidget()
    start_screen = StartScreen(stacked_widget)
    student_view = StudentView(stacked_widget)
    admin_panel = AdminPanel(stacked_widget)
    
    stacked_widget.addWidget(start_screen)    # index 0
    stacked_widget.addWidget(student_view)    # index 1
    stacked_widget.addWidget(admin_panel)
    
    stacked_widget.setCurrentIndex(0)
    stacked_widget.show()
    sys.exit(app.exec_())